In [1]:
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.common.by import By
df = pd.read_csv("kw_no_email_use_selenium.csv")

url_list = df['agent_url'].unique()
def get_driver():
    options = webdriver.FirefoxOptions()
    options.add_argument("--headless")
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-dev-shm-usage")
    driver = webdriver.Firefox(options=options)
    return driver


def collect_agent_contact_details(driver):
    try:
        agent_info = driver.find_element_by_class_name("AgentInformation")
    except:
        agent_info = ""
    try:
        info_container = agent_info.find_elements_by_class_name("AgentInformation__factBody")
    except:
        info_container = ""
    try:
        agent_email = [email.text for email in info_container if email.get_attribute("aria-label") == 'Agent E-mail'][0]
    except:
        agent_email = ""
    try:
        mobile_number = driver.find_element_by_class_name("AgentInformation__phoneMobileNumber").text
    except:
        mobile_number = ""
    try:
        office_number = driver.find_element_by_class_name("AgentInformation__phoneOfficeNumber").text
    except:
        office_number = ""
    try:
        agent_website = [i.text for i in info_container if i.get_attribute("aria-label") == 'Agent Website'][0]
    except:
        agent_website = ""
    contact_dict = {"agent_email":agent_email,"mobile_number":mobile_number,"office_number":office_number,'agent_website':agent_website }
    return contact_dict


def get_agent_details(driver):
    try:
        agent_licenses = driver.find_element_by_class_name("AgentContent__licenses").text
    except:
        agent_licenses = ""
    try:
        agent_bio = driver.find_element_by_class_name("AgentContent__bio").text
    except:
        agent_bio = ""
    try:
        serviceAreas = driver.find_element_by_class_name("AgentContent__serviceAreas").text
    except:
        serviceAreas = ""
    try:
        agent_team_name = driver.find_element_by_class_name("AgentContent__teamName").text
    except:
        agent_team_name = ""
    try:
        agent_team_info_ = driver.find_element_by_class_name("AgentContent__teamText").text.split("\n")
        agent_team_info = ", ".join(agent_team_info_)
    except:
        agent_team_info = ""
    try:
        logo_url = driver.find_element_by_class_name('AgentContent__teamAvatar').find_element_by_class_name("KWImage__image").get_attribute("src")
    except:
        logo_url = ""
    agent_details = {"agent_licenses":agent_licenses, "agent_bio":agent_bio, "serviceAreas":serviceAreas,"agent_team_name":agent_team_name,
                    "agent_team_info":agent_team_info,'logo_url':logo_url}
    return agent_details


def social_media_details(driver):
    try:
        social_media_container = driver.find_element_by_class_name("AgentInformation__socialMedia").find_elements_by_class_name("link")
    except:
        social_media_container = []
    social_media_links = [social_media.get_attribute("href") for social_media in social_media_container]
    try:
        facebook = [facebook for facebook in social_media_links if 'facebook' in facebook][0]
    except:
        facebook = ""
    try:
        instagram = [instagram for instagram in social_media_links if 'instagram' in instagram][0]
    except:
        instagram = ""
    try:
        twitter = [twitter for twitter in social_media_links if 'twitter' in twitter][0]
    except:
        twitter = ""
    try:
        linkedin = [linkedin for linkedin in social_media_links if 'linkedin' in linkedin][0]
    except:
        linkedin = ""
    social_media_dict = {"facebook":facebook,"instagram":instagram,"twitter":twitter,"linkedin":linkedin}
    social_media_links_str = "|".join(social_media_links)
    return social_media_dict,social_media_links_str


def collect_other_info(driver):
    container = driver.find_elements_by_class_name("AgentContent__section")
    try:
        Market_Cente = [i for i in container if 'Market Center' in i.text][0].find_element_by_class_name("AgentContent__sectionText").text
    except:
        Market_Cente = ""
    try:
        Languages = [i for i in container if 'Languages' in i.text][0].find_element_by_class_name("AgentContent__sectionText").text
    except:
        Languages = ""
    try:
        Specialties_designation = [i for i in container if 'Specialties and Designations' in i.text][0].find_element_by_class_name("AgentContent__sectionText").text
    except:
        Specialties_designation = ""
    other_info = {"Market_Cente":Market_Cente, "Languages":Languages, "Specialties_Designations":Specialties_designation}
    return other_info

def wait_to_page_load(driver):
    try:
        w = WebDriverWait(driver, 3)
        w.until(EC.presence_of_element_located((By.CLASS_NAME,"AgentContent__name")))
        print("Page load happened")
    except Exception as e:
        print("Timeout happened no page load")
    return driver

url_count = 37000
driver_count = 0
driver = get_driver()
for url in url_list[url_count:38000]:
    print(url)
    if driver_count >=100:
        print("need to close driver")
        driver.quit()
        driver_count = 0
        driver = get_driver()
        print("new driver initialize****************")
    driver.get(url)
    driver = wait_to_page_load(driver)
    try:
        agent_name = driver.find_element_by_class_name("AgentContent__name").text
    except:
        agent_name = ""
    try:
        time.sleep(0.5)
        profile_url = driver.find_element_by_class_name("AvatarImage__bg").get_attribute("style").replace('background-image: url("','').replace('");','')
    except:
        profile_url = ""
    try:
        tag = driver.find_element_by_class_name("pill").text
    except:
        tag = ""
    try:
        role = driver.find_element_by_class_name("AgentContent__team").text
    except:
        role = ""
    try:
        location = driver.find_element_by_class_name("AgentContent__location").text
    except:
        location = ""
    contact_dict = collect_agent_contact_details(driver)
    agent_details = get_agent_details(driver)
    social_media_dict,social_media_links_str = social_media_details(driver)
    other_info = collect_other_info(driver)
    data_dict = {'agent_url':url,"agent_name":agent_name, 'agent_role':role,'location':location,"agent_dp":profile_url, "tag":tag,"agent_email":contact_dict['agent_email'], "mobile_number":contact_dict['mobile_number'],"office_number":contact_dict['office_number'],'agent_license':agent_details['agent_licenses'], 'agent_bio':agent_details['agent_bio'],'serviceAreas':agent_details['serviceAreas'],'office_name':agent_details['agent_team_name'],'office_address':agent_details['agent_team_info'],'logo_url':agent_details['logo_url'],'facebook':social_media_dict['facebook'], 'instagram':social_media_dict['instagram'],'twitter':social_media_dict['twitter'],'linkedin':social_media_dict['linkedin'],'Market_Cente':other_info['Market_Cente'],'agent_language':other_info['Languages'],'Specialties_Designations':other_info['Specialties_Designations'],'social_media_links_str':social_media_links_str,'agent_website':contact_dict['agent_website']}
    data_df = pd.DataFrame(data_dict,index=[0],columns=['agent_url','agent_name', 'agent_role','location','agent_dp', 'tag','agent_email', 'mobile_number','office_number', 'agent_license', 'agent_bio', 'serviceAreas','office_name', 'office_address','logo_url','facebook', 'instagram', 'twitter', 'linkedin','Market_Cente', 'agent_language', 'Specialties_Designations','social_media_links_str','agent_website'])
    with open("kw_data-emailSelenium_37k_38k.csv",'a',newline='',encoding='utf-8') as f:
        data_df.to_csv(f, mode='a',header=f.tell()==0)
    print(url_count,url)
    url_count+=1
    driver_count+=1
    print("***********************************")
driver.quit()


https://kw.com/agent/UPA-6587385400092782595-9
Page load happened
37000 https://kw.com/agent/UPA-6587385400092782595-9
***********************************
https://kw.com/agent/UPA-6823274067479576576-8
Page load happened
37001 https://kw.com/agent/UPA-6823274067479576576-8
***********************************
https://kw.com/agent/UPA-6587385167899508739-2
Page load happened
37002 https://kw.com/agent/UPA-6587385167899508739-2
***********************************
https://kw.com/agent/UPA-6587385031262736386-0
Page load happened
37003 https://kw.com/agent/UPA-6587385031262736386-0
***********************************
https://kw.com/agent/UPA-6790343354731438080-6
Page load happened
37004 https://kw.com/agent/UPA-6790343354731438080-6
***********************************
https://kw.com/agent/UPA-6803796278197702656-0
Page load happened
37005 https://kw.com/agent/UPA-6803796278197702656-0
***********************************
https://kw.com/agent/UPA-6770743911820337152-0
Page load happened
3700

Page load happened
37053 https://kw.com/agent/UPA-6699065103250685952-4
***********************************
https://kw.com/agent/UPA-6587385160296914950-0
Page load happened
37054 https://kw.com/agent/UPA-6587385160296914950-0
***********************************
https://kw.com/agent/UPA-6587385432157917189-3
Page load happened
37055 https://kw.com/agent/UPA-6587385432157917189-3
***********************************
https://kw.com/agent/UPA-6592998104294268928-2
Page load happened
37056 https://kw.com/agent/UPA-6592998104294268928-2
***********************************
https://kw.com/agent/UPA-6587385273883201541-1
Page load happened
37057 https://kw.com/agent/UPA-6587385273883201541-1
***********************************
https://kw.com/agent/UPA-6800051385771204608-1
Page load happened
37058 https://kw.com/agent/UPA-6800051385771204608-1
***********************************
https://kw.com/agent/UPA-6682425498512494592-7
Page load happened
37059 https://kw.com/agent/UPA-6682425498512494592-

Page load happened
37106 https://kw.com/agent/UPA-6587385236992593927-5
***********************************
https://kw.com/agent/UPA-6837060380146888704-3
Page load happened
37107 https://kw.com/agent/UPA-6837060380146888704-3
***********************************
https://kw.com/agent/UPA-6780558272217858048-4
Page load happened
37108 https://kw.com/agent/UPA-6780558272217858048-4
***********************************
https://kw.com/agent/UPA-6623280283685781505-4
Page load happened
37109 https://kw.com/agent/UPA-6623280283685781505-4
***********************************
https://kw.com/agent/UPA-6587384994501251077-6
Page load happened
37110 https://kw.com/agent/UPA-6587384994501251077-6
***********************************
https://kw.com/agent/UPA-6587385156974682112-3
Page load happened
37111 https://kw.com/agent/UPA-6587385156974682112-3
***********************************
https://kw.com/agent/UPA-6683482815258624000-3
Page load happened
37112 https://kw.com/agent/UPA-6683482815258624000-

Page load happened
37159 https://kw.com/agent/UPA-6587385148825919489-6
***********************************
https://kw.com/agent/UPA-6697646694232174592-6
Page load happened
37160 https://kw.com/agent/UPA-6697646694232174592-6
***********************************
https://kw.com/agent/UPA-6587385453103271937-9
Page load happened
37161 https://kw.com/agent/UPA-6587385453103271937-9
***********************************
https://kw.com/agent/UPA-6813881681364361216-8
Page load happened
37162 https://kw.com/agent/UPA-6813881681364361216-8
***********************************
https://kw.com/agent/UPA-6638481390084186112-0
Page load happened
37163 https://kw.com/agent/UPA-6638481390084186112-0
***********************************
https://kw.com/agent/UPA-6761065251530211328-6
Page load happened
37164 https://kw.com/agent/UPA-6761065251530211328-6
***********************************
https://kw.com/agent/UPA-6771120611391418368-4
Page load happened
37165 https://kw.com/agent/UPA-6771120611391418368-

Page load happened
37212 https://kw.com/agent/UPA-6587385436123914240-5
***********************************
https://kw.com/agent/UPA-6587384961338064896-4
Timeout happened no page load
37213 https://kw.com/agent/UPA-6587384961338064896-4
***********************************
https://kw.com/agent/UPA-6587385229311401990-4
Page load happened
37214 https://kw.com/agent/UPA-6587385229311401990-4
***********************************
https://kw.com/agent/UPA-6592930436279234567-0
Timeout happened no page load
37215 https://kw.com/agent/UPA-6592930436279234567-0
***********************************
https://kw.com/agent/UPA-6806619497754247168-9
Page load happened
37216 https://kw.com/agent/UPA-6806619497754247168-9
***********************************
https://kw.com/agent/UPA-6757381215038889984-8
Page load happened
37217 https://kw.com/agent/UPA-6757381215038889984-8
***********************************
https://kw.com/agent/UPA-6780980340778274816-6
Page load happened
37218 https://kw.com/agent/UP

...

Page load happened
37742 https://kw.com/agent/UPA-6793589092856500224-2
***********************************
https://kw.com/agent/UPA-6587385173112492037-1
Page load happened
37743 https://kw.com/agent/UPA-6587385173112492037-1
***********************************
https://kw.com/agent/UPA-6587385239231250432-4
Page load happened
37744 https://kw.com/agent/UPA-6587385239231250432-4
***********************************
https://kw.com/agent/UPA-6587385031468257281-3
Page load happened
37745 https://kw.com/agent/UPA-6587385031468257281-3
***********************************
https://kw.com/agent/UPA-6587385408027070471-2
Page load happened
37746 https://kw.com/agent/UPA-6587385408027070471-2
***********************************
https://kw.com/agent/UPA-6587385160775065603-3
Page load happened
37747 https://kw.com/agent/UPA-6587385160775065603-3
***********************************
https://kw.com/agent/UPA-6587385170587713542-8
Page load happened
37748 https://kw.com/agent/UPA-6587385170587713542-

Page load happened
37795 https://kw.com/agent/UPA-6587385193325973507-9
***********************************
https://kw.com/agent/UPA-6780255396658577408-7
Page load happened
37796 https://kw.com/agent/UPA-6780255396658577408-7
***********************************
https://kw.com/agent/UPA-6587385154548879360-3
Page load happened
37797 https://kw.com/agent/UPA-6587385154548879360-3
***********************************
https://kw.com/agent/UPA-6587385279788085248-7
Page load happened
37798 https://kw.com/agent/UPA-6587385279788085248-7
***********************************
https://kw.com/agent/UPA-6587385066141777922-8
Page load happened
37799 https://kw.com/agent/UPA-6587385066141777922-8
***********************************
https://kw.com/agent/UPA-6677232274144620544-1
need to close driver
new driver initialize****************
Page load happened
37800 https://kw.com/agent/UPA-6677232274144620544-1
***********************************
https://kw.com/agent/UPA-6587384978719936515-1
Page load h

Page load happened
37848 https://kw.com/agent/UPA-6755568981930369024-8
***********************************
https://kw.com/agent/UPA-6587385180162867201-6
Page load happened
37849 https://kw.com/agent/UPA-6587385180162867201-6
***********************************
https://kw.com/agent/UPA-6754481812689313792-4
Page load happened
37850 https://kw.com/agent/UPA-6754481812689313792-4
***********************************
https://kw.com/agent/UPA-6696120743399780352-7
Page load happened
37851 https://kw.com/agent/UPA-6696120743399780352-7
***********************************
https://kw.com/agent/UPA-6676250528222052352-9
Page load happened
37852 https://kw.com/agent/UPA-6676250528222052352-9
***********************************
https://kw.com/agent/UPA-6587385254712451079-7
Page load happened
37853 https://kw.com/agent/UPA-6587385254712451079-7
***********************************
https://kw.com/agent/UPA-6608309913764990978-7
Page load happened
37854 https://kw.com/agent/UPA-6608309913764990978-

Page load happened
37901 https://kw.com/agent/UPA-6805999610484961280-9
***********************************
https://kw.com/agent/UPA-6818275493368262656-8
Page load happened
37902 https://kw.com/agent/UPA-6818275493368262656-8
***********************************
https://kw.com/agent/UPA-6701543100258189312-3
Page load happened
37903 https://kw.com/agent/UPA-6701543100258189312-3
***********************************
https://kw.com/agent/UPA-6587385219851837445-2
Page load happened
37904 https://kw.com/agent/UPA-6587385219851837445-2
***********************************
https://kw.com/agent/UPA-6805199886182670336-2
Page load happened
37905 https://kw.com/agent/UPA-6805199886182670336-2
***********************************
https://kw.com/agent/UPA-6587385440993521668-4
Page load happened
37906 https://kw.com/agent/UPA-6587385440993521668-4
***********************************
https://kw.com/agent/UPA-6834583315957862400-0
Timeout happened no page load
37907 https://kw.com/agent/UPA-683458331

Page load happened
37954 https://kw.com/agent/UPA-6587385260241285121-9
***********************************
https://kw.com/agent/UPA-6674377652724228096-4
Page load happened
37955 https://kw.com/agent/UPA-6674377652724228096-4
***********************************
https://kw.com/agent/UPA-6783472639829798912-1
Page load happened
37956 https://kw.com/agent/UPA-6783472639829798912-1
***********************************
https://kw.com/agent/UPA-6587385179181936646-0
Page load happened
37957 https://kw.com/agent/UPA-6587385179181936646-0
***********************************
https://kw.com/agent/UPA-6587385222350336005-4
Page load happened
37958 https://kw.com/agent/UPA-6587385222350336005-4
***********************************
https://kw.com/agent/UPA-6587385151243821063-4
Page load happened
37959 https://kw.com/agent/UPA-6587385151243821063-4
***********************************
https://kw.com/agent/UPA-6587385386916184070-4
Timeout happened no page load
37960 https://kw.com/agent/UPA-658738538